In [1]:
import os

import anthropic
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build


SCOPES = ['https://www.googleapis.com/auth/calendar']



if os.path.exists('./res/token.json'):
    creds = Credentials.from_authorized_user_file('./res/token.json', SCOPES)
else:
    flow = InstalledAppFlow.from_client_secrets_file(
        './res/credentials.json', SCOPES
    )
    creds = flow.run_local_server(port=0)
    with open('./res/token.json', 'w') as token:
        token.write(creds.to_json())

service = build('calendar', 'v3', credentials=creds)

### Function Call (Tool Use) 복습

In [2]:
client = anthropic.Anthropic(
    api_key=os.environ['ANTHROPIC_API_KEY']
)

response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    tools=[
        {
            "name": "get_weather",
            "description": "Get the current weather in a given location",
            "input_schema": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    }
                },
                "required": ["location"],
            },
        }
    ],
    messages=[{"role": "user", "content": "What's the weather like in San Francisco?"}],
)

In [3]:
response.content

[TextBlock(text='Certainly! I can help you with that information. Let me check the current weather in San Francisco for you using the available weather tool.', type='text'),
 ToolUseBlock(id='toolu_01PqFKZJaY3dCjawydorEwLW', input={'location': 'San Francisco, CA'}, name='get_weather', type='tool_use')]

### Function Call + Google Calendar API

In [4]:
event = {
  'summary': '패스트캠퍼스 LLM 강의',
  'location': '서울 강남구 테헤란로 231 West동 6층, 7층',
  'start': {
    'dateTime': '2024-08-14T09:00:00+09:00',
    'timeZone': 'Asia/Seoul',
  },
  'end': {
    'dateTime': '2024-08-14T18:00:00+09:00',
    'timeZone': 'Asia/Seoul',
  }
}

In [5]:
tools = [
    {
        "name": "create_event",
        "description": "Create new Google Calender Event",
        "input_schema": {
            "type": "object",
            "properties": {
                "summary": {
                    "type": "string",
                    "description": "Name of Google Calender Event"
                },
                "start": {
                    "type": "string",
                    "description": "Starting date of Google Calender Event in UTC+9 Time i.e. 2024-08-08T09:00:00+09:00"
                },
                "end": {
                    "type": "string",
                    "description": "Starting date of Google Calender Event in UTC+9 i.e. 2024-08-08T09:00:00+09:00"
                }
            },
            "required": ["summary"]
        }
    }
]

In [6]:
def create_event(summary, start, end):
    event = {
        'summary': summary,
        'start': {
            'dateTime': start,
            'timeZone': 'Asia/Seoul',
        },
        'end': {
            'dateTime': end,
            'timeZone': 'Asia/Seoul',
        }
    }
    event = service.events().insert(calendarId='primary', body=event).execute()
    print('Event created: %s' % (event.get('htmlLink')))
    return event

In [7]:
response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    tools=tools,
    messages=[
        {
            "role": "user",
            "content": "Create a calender event called Lunch starting from 2024.08.14 1pm"
        }
    ],
)

In [8]:
response.content

[TextBlock(text='Certainly! I\'d be happy to help you create a calendar event for your lunch. I\'ll use the "create_event" function to set this up for you. However, I\'ll need to clarify a couple of details to ensure the event is created correctly.\n\nFirst, I have the summary (name) of the event and the start time, but I need to know how long you\'d like to allocate for lunch so I can set an end time. Typically, lunch events are scheduled for an hour, but please let me know if you\'d prefer a different duration.\n\nAlso, the function requires the time to be in UTC+9 format. I\'ll assume you\'re in the same time zone, but please correct me if I\'m wrong.\n\nBased on the information you\'ve provided and assuming a one-hour duration, here\'s how I would create the event:', type='text'),
 ToolUseBlock(id='toolu_016SdYeUboAXtQ7zSkREER7g', input={'summary': 'Lunch', 'start': '2024-08-14T13:00:00+09:00', 'end': '2024-08-14T14:00:00+09:00'}, name='create_event', type='tool_use')]

In [9]:
response.content[1].input

{'summary': 'Lunch',
 'start': '2024-08-14T13:00:00+09:00',
 'end': '2024-08-14T14:00:00+09:00'}

In [10]:
summary = response.content[1].input['summary']
start = response.content[1].input['start']
end = response.content[1].input['end']
create_event(summary, start, end)

Event created: https://www.google.com/calendar/event?eid=NDExc2Via3BuYmFnOThybzBybzM2dGljc2MgcmVpbjIwLmFpQG0


{'kind': 'calendar#event',
 'etag': '"3447273218704000"',
 'id': '411sebkpnbag98ro0ro36ticsc',
 'status': 'confirmed',
 'htmlLink': 'https://www.google.com/calendar/event?eid=NDExc2Via3BuYmFnOThybzBybzM2dGljc2MgcmVpbjIwLmFpQG0',
 'created': '2024-08-14T11:56:49.000Z',
 'updated': '2024-08-14T11:56:49.352Z',
 'summary': 'Lunch',
 'creator': {'email': 'rein20.ai@gmail.com', 'self': True},
 'organizer': {'email': 'rein20.ai@gmail.com', 'self': True},
 'start': {'dateTime': '2024-08-14T13:00:00+09:00', 'timeZone': 'Asia/Seoul'},
 'end': {'dateTime': '2024-08-14T14:00:00+09:00', 'timeZone': 'Asia/Seoul'},
 'iCalUID': '411sebkpnbag98ro0ro36ticsc@google.com',
 'sequence': 0,
 'reminders': {'useDefault': True},
 'eventType': 'default'}

### End-to-End Function Call Workflow
1. Function 준비 (또는 작성)
2. Tools 정의
3. LLM API 호출
4. (필요 시) Function 실행 - 일반 발화가 필요한경우 실행 필요 X
5. LLM API 최종 호출

In [11]:
PROMPT = '2024년 8월 14일 12시부터 1시간 짜리 점심 일정 생성해주세요.'

# 1. 함수 생성
def create_event(summary, start, end):
    event = {
        'summary': summary,
        'start': {
            'dateTime': start,
            'timeZone': 'Asia/Seoul',
        },
        'end': {
            'dateTime': end,
            'timeZone': 'Asia/Seoul',
        }
    }
    event = service.events().insert(calendarId='primary', body=event).execute()
    print('Event created: %s' % (event.get('htmlLink')))
    return event

# 2. tools 정의
tools = [
    {
        "name": "create_event",
        "description": "Create new Google Calender Event",
        "input_schema": {
            "type": "object",
            "properties": {
                "summary": {
                    "type": "string",
                    "description": "Name of Google Calender Event"
                },
                "start": {
                    "type": "string",
                    "description": "Starting date of Google Calender Event in UTC+9 Time i.e. 2024-08-08T09:00:00+09:00"
                },
                "end": {
                    "type": "string",
                    "description": "Starting date of Google Calender Event in UTC+9 Time i.e. 2024-08-08T09:00:00+09:00"
                }
            },
            "required": ["summary"]
        }
    }
]

# 3. LLM API 호출
response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    tools=tools,
    messages=[
        {
            "role": "user",
            "content": PROMPT
        }
    ],
)

# 4. (필요 시) Function Call 실행
summary = response.content[1].input['summary']
start = response.content[1].input['start']
end = response.content[1].input['end']
event = create_event(summary, start, end)

# 5. LLM API 최종 호출
response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    tools=tools,
    messages=[
        {
            "role": "user",
            "content": PROMPT
        },
        {
            "role": "assistant",
            "content": [
                {
                    "type": "text",
                    "text": PROMPT
                },
                {
                    "type": "tool_use",
                    "id": response.content[1].id,
                    "name": "create_event",
                    "input": {"summary": summary, "start": start, "end": end}
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": response.content[1].id,
                    "content": f"Event created: {event.get('htmlLink')}"
                }
            ]
        }
    ],
)

Event created: https://www.google.com/calendar/event?eid=NDhiODUxMTZpN3VnbTY0ZDZkMTRycms3MDAgcmVpbjIwLmFpQG0


In [12]:
print(response.content[0].text)

네, 2024년 8월 14일 12시부터 1시간 동안의 점심 일정을 생성했습니다. Google 캘린더에 다음과 같이 일정이 추가되었습니다:

- 일정 제목: 점심 일정
- 시작 시간: 2024년 8월 14일 오후 12:00
- 종료 시간: 2024년 8월 14일 오후 1:00

일정이 성공적으로 생성되었습니다. 필요하시다면 제공된 링크를 통해 일정을 확인하거나 수정하실 수 있습니다. 다른 도움이 필요하신가요?
